# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [15]:
# #langchain library
# !pip install langchain==0.1.0
# #LLM
# !pip install accelerate==0.25.0
# !pip install transformers==4.36.2
# !pip install bitsandbytes==0.41.2
# #Text Embedding
# !pip install sentence-transformers==2.2.2
# !pip install InstructorEmbedding==1.0.1
# #vectorstore
# !pip install pymupdf==1.23.8
# !pip install faiss-gpu==1.7.2
# !pip install faiss-cpu==1.7.4

In [16]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [17]:
from langchain import PromptTemplate

prompt_template = """
    I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format 
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:")

In [18]:
PROMPT.format(
    context = "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions.",
    question = "What is Machine Learning"
)

"I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions.\n    Question: What is Machine Learning\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code). 
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders 
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [19]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = './resources/book.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [20]:
# documents

In [21]:
len(documents)

577

In [22]:
documents[1]

Document(page_content='Summary of Contents\nI\nFundamental Algorithms for NLP\n1\n1\nIntroduction. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n3\n2\nRegular Expressions, Text Normalization, Edit Distance. . . . . . . . .\n4\n3\nN-gram Language Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n32\n4\nNaive Bayes, Text Classiﬁcation, and Sentiment . . . . . . . . . . . . . . . . . 60\n5\nLogistic Regression . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 81\n6\nVector Semantics and Embeddings . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 105\n7\nNeural Networks and Neural Language Models . . . . . . . . . . . . . . . . . 136\n8\nSequence Labeling for Parts of Speech and Named Entities . . . . . . 162\n9\nRNNs and LSTMs . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 187\n10 Transformers and Large Language Mo

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [24]:
doc[1]

Document(page_content='Summary of Contents\nI\nFundamental Algorithms for NLP\n1\n1\nIntroduction. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n3\n2\nRegular Expressions, Text Normalization, Edit Distance. . . . . . . . .\n4\n3\nN-gram Language Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n32\n4\nNaive Bayes, Text Classiﬁcation, and Sentiment . . . . . . . . . . . . . . . . . 60\n5\nLogistic Regression . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 81\n6\nVector Semantics and Embeddings . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 105\n7', metadata={'source': './resources/book.pdf', 'file_path': './resources/book.pdf', 'page': 1, 'total_pages': 577, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.21', 'creationDate': "D:20240203145732-08'00'", 'modDate':

In [25]:
len(doc)

3102

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [26]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

load INSTRUCTOR_Transformer
max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [27]:
#locate vectorstore
vector_path = './vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

create path done


In [28]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [29]:
#calling vector from local
# vector_path = './vector-store'
# db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)   

In [30]:
#ready to use
retriever = vectordb.as_retriever()

In [31]:
retriever.get_relevant_documents("What is Dependency Parsing")

[Document(page_content='Dependency parsing saw a major resurgence in the late 1990’s with the appear-\nance of large dependency-based treebanks and the associated advent of data driven\napproaches described in this chapter. Eisner (1996) developed an efﬁcient dynamic\nprogramming approach to dependency parsing based on bilexical grammars derived\nfrom the Penn Treebank. Covington (2001) introduced the deterministic word by\nword approach underlying current transition-based approaches. Yamada and Mat-\nsumoto (2003) and Kudo and Matsumoto (2002) introduced both the shift-reduce\nparadigm and the use of supervised machine learning in the form of support vector\nmachines to dependency parsing.', metadata={'source': './resources/book.pdf', 'file_path': './resources/book.pdf', 'page': 420, 'total_pages': 577, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.21', 'creationDate': "D:20240203145732-08'00'"

In [32]:
retriever.get_relevant_documents("What is Transformers")

[Document(page_content='BIBLIOGRAPHICAL AND HISTORICAL NOTES\n241\n• Transformers are non-recurrent networks based on self-attention. A self-\nattention layer maps input sequences to output sequences of the same length,\nusing attention heads that model how the surrounding words are relevant for\nthe processing of the current word.\n• A transformer block consists of a single attention layer followed by a feed-\nforward layer with residual connections and layer normalizations following\neach. Transformer blocks can be stacked to make deeper and more powerful\nnetworks.\n• Language models can be built out of stacks of transformer blocks, with a linear\nand softmax max layer at the top.', metadata={'source': './resources/book.pdf', 'file_path': './resources/book.pdf', 'page': 248, 'total_pages': 577, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.21', 'creationDate': "D:20240203145732-08'00'", 'modD

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [33]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [34]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [35]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios. 
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [36]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [37]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time. 
- it only uses the last K interactions. 
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [38]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available), 
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [39]:
# %cd ./models
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

In [40]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = './resources/fastchat-t5-3b-v1.0/'

tokenizer = AutoTokenizer.from_pretrained("lmsys/fastchat-t5-3b-v1.0")

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = False,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    "lmsys/fastchat-t5-3b-v1.0",
    # quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'mps',
    load_in_8bit = False
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [41]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [42]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [43]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [44]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  are  the  main  differences  between  Machine  Learning  and  Deep  Learning  AI?\n'}

`combine_docs_chain`

In [45]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x2921d6090>)), document_variable_name='context')

In [46]:
query = "What is Transformers?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    BIBLIOGRAPHICAL AND HISTORICAL NOTES
241
• Transformers are non-recurrent networks based on self-attention. A self-
attention layer maps input sequences to output sequences of the same length,
using attention heads that model how the surrounding words are relevant for
the processing of the current word.
• A transformer block consists of a single atte

{'input_documents': [Document(page_content='BIBLIOGRAPHICAL AND HISTORICAL NOTES\n241\n• Transformers are non-recurrent networks based on self-attention. A self-\nattention layer maps input sequences to output sequences of the same length,\nusing attention heads that model how the surrounding words are relevant for\nthe processing of the current word.\n• A transformer block consists of a single attention layer followed by a feed-\nforward layer with residual connections and layer normalizations following\neach. Transformer blocks can be stacked to make deeper and more powerful\nnetworks.\n• Language models can be built out of stacks of transformer blocks, with a linear\nand softmax max layer at the top.', metadata={'source': './resources/book.pdf', 'file_path': './resources/book.pdf', 'page': 248, 'total_pages': 577, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.21', 'creationDate': "D:202402031

In [47]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2Te

## 5. Chatbot

In [48]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    EXERCISES
59
3.3
Which of the two probabilities you computed in the previous exercise is higher,
unsmoothed or smoothed? Explain why.
3.4
We are given the following corpus, modiﬁed from the one in the chapter:
<s> I am Sam </s>
<s> Sam I am </s>
<s> I am Sam </s>
<s> I do not like green eggs and 

{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': '<pad>  I  am  Sam\n Answer:  I  am  Sam\n 3.5  Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol  :\n< s>  I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam',
 'source_documents': [Document(page_content='EXERCISES\n59\n3.3\nWhich of the two probabilities you computed in the previous exercise is higher,\nunsmoothed or smoothed? Explain why.\n3.4\nWe are given the following corpus, modiﬁed from the one in the chapter:\n<s> I am Sam </s>\n<s> Sam I am </s>\n<s> I am Sam </s>\n<s> I do not like green eggs an

In [49]:
prompt_question = "What is the Transformers?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad>  I  am  Sam\n Answer:  I  am  Sam\n 3.5  Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol  :\n< s>  I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam')]
Follow Up Input: What is the Transformers?
Standalone question:

> Finished c

{'question': 'What is the Transformers?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad>  I  am  Sam\n Answer:  I  am  Sam\n 3.5  Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol  :\n< s>  I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam')],
 'answer': '<pad>  transformers  are  a  neural  architecture  that  can  handle  distant  information.  They  are  non-recurrent  networks  based  on  self-attention.  A  self-attention  layer  maps  input  sequences  to  output  sequences  of  the  same  length,  using  attention  heads  

In [50]:
prompt_question = "Is it a statistical model?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad>  I  am  Sam\n Answer:  I  am  Sam\n 3.5  Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol  :\n< s>  I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam'), HumanMessage(content='What is the Transformers?'), AIMessage(content='<pad>  

{'question': 'Is it a statistical model?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad>  I  am  Sam\n Answer:  I  am  Sam\n 3.5  Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol  :\n< s>  I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam\n I  am  Sam'),
  HumanMessage(content='What is the Transformers?'),
  AIMessage(content='<pad>  transformers  are  a  neural  architecture  that  can  handle  distant  information.  They  are  non-recurrent  networks  based  on  self-attention.  A  self-attention  layer  maps  input  sequences  to  output